**Análise de Degradação da Floresta Amazônica (2004-2019)**

Este projeto faz parte da disciplina de Big Data, lecionada pelo professor e Msc. Sebastião Rogério, onde nela temos um projeto cujo objetivo é realizar uma análise de dados focada na degradação da Floresta Amazônica entre os anos de 2004 e 2019. Utilizando o Spark para processamento distribuído e integração de diferentes fontes de dados, buscamos correlacionar fatores ambientais, econômicos e sociais com o desmatamento na Amazônia Legal.

**Objetivo**

O projeto visa explorar e analisar padrões de desmatamento, identificar as principais causas da degradação e compreender o impacto ambiental em diferentes estados brasileiros. Também propomos correlações entre as áreas desmatadas e fatores como crescimento agrícola, urbanização e condições climáticas, utilizando técnicas de análise de dados avançadas e aprendizado de máquina.

**Tecnologias Utilizadas**

**Apache Spark:** Para processamento de grandes volumes de dados e implementação de modelos de machine learning.
Google Colaboratory: Ambiente de desenvolvimento para o projeto, permitindo a execução de análises e visualizações em escala.
**Python (PySpark):** Ferramenta principal para manipulação e análise de dados.
Bibliotecas de Visualização: Utilizaremos bibliotecas como Matplotlib, Seaborn e Plotly para criar gráficos interativos e visuais que destacam os insights descobertos.

**Dados Utilizados**

O dataset principal utilizado nesta análise é o "Brazilian Amazon Rainforest Degradation 1999-2019", disponível no Kaggle, que contém informações detalhadas sobre as áreas desmatadas em diversos estados da Amazônia Legal, com foco nas causas naturais e antropogênicas de degradação, como agricultura ilegal, mineração, queimadas e expansão urbana.

**Resultados Esperados**

Esperamos identificar as tendências e padrões no desmatamento, além de sugerir possíveis intervenções para mitigar os efeitos nocivos da degradação ambiental na região. O projeto também inclui o desenvolvimento de um dashboard interativo, que permitirá aos usuários explorar os dados e insights obtidos de maneira dinâmica e visual.

In [ ]:
# Configurar o ambiente
!pip install plotly
!pip install pyspark
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Agricultural and Deforestation Analysis").getOrCreate()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Carregar os datasets no Spark
df_deforestation_spark = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/BigData/def_area_2004_2019.csv', header=True, inferSchema=True)
df_agriculture_spark = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/BigData/harvested_area.csv', header=True, inferSchema=True)

# Exibir as primeiras linhas
df_deforestation_spark.show(5)
df_agriculture_spark.show(5)

+-----------+---+----+---+----+-----+----+----+---+---+---------+
|Ano/Estados| AC|  AM| AP|  MA|   MT|  PA|  RO| RR| TO|AMZ LEGAL|
+-----------+---+----+---+----+-----+----+----+---+---+---------+
|       2004|728|1232| 46| 755|11814|8870|3858|311|158|    27772|
|       2005|592| 775| 33| 922| 7145|5899|3244|133|271|    19014|
|       2006|398| 788| 30| 674| 4333|5659|2049|231|124|    14286|
|       2007|184| 610| 39| 631| 2678|5526|1611|309| 63|    11651|
|       2008|254| 604|100|1271| 3258|5607|1136|574|107|    12911|
+-----------+---+----+---+----+-----+----+----+---+---+---------+
only showing top 5 rows

+---+-------+----+--------------------------+--------------+-----------+-----------+---------------+----------------------+---------------+------------------+--------------------+------------------+---------+--------------+-----------------------+-----------------------------+------------------------+------------------+----------------+----------------------+---------------+----

In [ ]:
# Estatísticas básicas com Spark
df_deforestation_spark.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+
|summary|      Ano/Estados|                AC|                AM|                AP|                MA|                MT|               PA|               RO|                RR|                TO|        AMZ LEGAL|
+-------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+
|  count|               16|                16|                16|                16|                16|                16|               16|               16|                16|                16|               16|
|   mean|           2011.5|           357.625|          776.5625|              38.5|           519.875|         2691.5625|         3923.625|

In [ ]:
# Renomear a coluna 'Ano/Estados' para 'Year' e remover o espaço de 'AMZ LEGAL'
df_deforestation_spark = df_deforestation_spark.withColumnRenamed('Ano/Estados', 'Year') \
                                               .withColumnRenamed('AMZ LEGAL', 'AMZ_LEGAL')

# Coletar dados necessários para o gráfico de desmatamento
df_deforestation_pandas = df_deforestation_spark.select('Year', 'AMZ_LEGAL').toPandas()


fig = px.bar(
    df_deforestation_pandas,
    x='Year',
    y='AMZ_LEGAL',
    title='Desmatamento na Amazônia Legal (2004-2019)',
    labels={'Year': 'Ano', 'AMZ_LEGAL': 'Área desmatada (km²)'},
    color='AMZ_LEGAL',
    color_continuous_scale="Blues"
)
fig.update_layout(
    xaxis_title='Ano',
    yaxis_title='Área desmatada (km²)',
    xaxis=dict(tickangle=45),
    coloraxis_showscale=False
)
fig.show()

In [ ]:
# Coletar dados para comparação entre estados
df_states_pandas = df_deforestation_spark.select('Year', 'AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO').toPandas()

fig = px.line(
    df_states_pandas,
    x='Year',
    y=['AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO'],
    title='Evolução do desmatamento por Estado',
    labels={'Year': 'Ano', 'value': 'Área desmatada (km²)', 'variable': 'Estado'}
)

fig.update_layout(
    xaxis_title='Ano',
    yaxis_title='Área desmatada (km²)',
)

fig.show()


**A partir daqui, veremos as tendências, como padrões temporais, picos, quedas, comparações entre os estados utilizando heatmaps**

In [ ]:
# Coletar dados necessários para o gráfico de tendência
df_tendencia_pandas = df_deforestation_spark.select('Year', 'AMZ_LEGAL').toPandas()

fig = px.line(
    df_tendencia_pandas,
    x='Year',
    y='AMZ_LEGAL',
    title='Tendência do desmatamento na Amazônia Legal (2004-2019)',
    labels={'Year': 'Ano', 'AMZ_LEGAL': 'Área desmatada (km²)'},
    markers=True
)

fig.update_layout(
    xaxis_title='Ano',
    yaxis_title='Área desmatada (km²)',
)

fig.show()


### Correlação entre os Datasets: Desmatamento da Floresta Amazônica (2004-2019) e Produção Agrícola no Brasil

A partir deste ponto, será realizada a correlação entre dois datasets:

1. **Brazilian Amazon Rainforest Degradation (2004-2019)**: Este dataset contém informações sobre o desmatamento da Floresta Amazônica no Brasil, destacando as causas principais, como a agricultura ilegal, desastres naturais, urbanização, mineração e incêndios. Ele também inclui dados anuais de desmatamento por estado na Amazônia Legal.

2. **Brazilian Agricultural Production (1974-Presente)**: Este segundo dataset compila dados sobre a produção de diversos grãos no Brasil, incluindo soja, milho, sorgo, trigo, feijão, aveia, entre outros. As informações estão organizadas por município e incluem a área plantada (hectares), produção (toneladas) e rendimento (toneladas por hectare). Para este projeto, os dados foram filtrados entre 2004 e 2019 e agregados por estado, a fim de facilitar a correlação com o desmatamento.

Nos próximos passos, será investigada a relação entre o aumento da área agrícola e o desmatamento em diferentes estados, buscando identificar padrões que indiquem a influência da expansão agrícola sobre a degradação da Amazônia.


In [ ]:
# Verificar os nomes das colunas do dataset agrícola no Spark
df_agriculture_spark.columns

['_c0',
 'Grain',
 'Year',
 "Alta Floresta D'Oeste (RO)",
 'Ariquemes (RO)',
 'Cabixi (RO)',
 'Cacoal (RO)',
 'Cerejeiras (RO)',
 'Colorado do Oeste (RO)',
 'Corumbiara (RO)',
 'Costa Marques (RO)',
 "Espigão D'Oeste (RO)",
 'Guajará-Mirim (RO)',
 'Jaru (RO)',
 'Ji-Paraná (RO)',
 "Machadinho D'Oeste (RO)",
 "Nova Brasilândia D'Oeste (RO)",
 'Ouro Preto do Oeste (RO)',
 'Pimenta Bueno (RO)',
 'Porto Velho (RO)',
 'Presidente Médici (RO)',
 'Rio Crespo (RO)',
 'Rolim de Moura (RO)',
 "Santa Luzia D'Oeste (RO)",
 'Vilhena (RO)',
 'São Miguel do Guaporé (RO)',
 'Nova Mamoré (RO)',
 "Alvorada D'Oeste (RO)",
 'Alto Alegre dos Parecis (RO)',
 'Alto Paraíso (RO)',
 'Buritis (RO)',
 'Novo Horizonte do Oeste (RO)',
 'Cacaulândia (RO)',
 'Campo Novo de Rondônia (RO)',
 'Candeias do Jamari (RO)',
 'Castanheiras (RO)',
 'Chupinguaia (RO)',
 'Cujubim (RO)',
 'Governador Jorge Teixeira (RO)',
 'Itapuã do Oeste (RO)',
 'Ministro Andreazza (RO)',
 'Mirante da Serra (RO)',
 'Monte Negro (RO)',
 'Nova Un

Ao analisar as colunas do nosso dataframe, percebemos que não conseguimos dividir os dados por estado e ano, isso porquê não existem colunas que dividem esses dados, uma vez que as colunas na verdade são as cidades, dificultando para correlacionarmos com o primeiro dataset. Teremos que ajustar ele em dois passos:

**Converter as colunas dos municípios em linhas**;

**Adicionar uma coluna de estado:**

Teremos que identificar a qual estado cada município pertence, pois os nomes das cidades não estão associados diretamente com estados.

In [ ]:
from pyspark.sql.functions import col

# Coletar os nomes das colunas que representam os municípios (excluindo as colunas 'Grain' e 'Year')
municipality_columns = df_agriculture_spark.columns[2:]  # Assumindo que as duas primeiras colunas são 'Grain' e 'Year'

# Converter todas as colunas de municípios para numérico (caso haja algum erro de tipo)
df_agriculture_spark = df_agriculture_spark.select(
    [col(c).cast("float").alias(c) if c in municipality_columns else col(c) for c in df_agriculture_spark.columns]
)

# Função para tratar apóstrofos em nomes de municípios
def escape_apostrophe(municipality):
    return municipality.replace("'", "\\'")

# Construir a expressão para a função stack dinamicamente, escapando os apóstrofos
stack_expr = "stack({}, {}) as (Municipality, Production)".format(
    len(municipality_columns),
    ', '.join([f"'{escape_apostrophe(municipality)}', `{municipality}`" for municipality in municipality_columns])
)

# Aplicar a função stack com as colunas de municípios, excluindo 'Year'
df_agriculture_melted_spark = df_agriculture_spark.selectExpr("Year", stack_expr)

# Filtrar as linhas onde o Município não é 'Year'
df_agriculture_melted_spark = df_agriculture_melted_spark.filter(df_agriculture_melted_spark.Municipality != "Year")

# Mostrar as primeiras linhas (Produção maior que 0)
df_agriculture_melted_spark.filter(df_agriculture_melted_spark.Production > 0).show(5)

+------+--------------------+----------+
|  Year|        Municipality|Production|
+------+--------------------+----------+
|1974.0|Fortaleza dos Nog...|       4.0|
|1974.0|        Riachão (MA)|       6.0|
|1974.0|     Maranguape (CE)|      80.0|
|1974.0|            Açu (RN)|      30.0|
|1974.0|   Areia Branca (RN)|     350.0|
+------+--------------------+----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import split

# Adicionar a coluna de Estado ao DataFrame derretido, extraindo a sigla do estado dos nomes dos municípios
df_agriculture_with_state = df_agriculture_melted_spark.withColumn('State', split(col('Municipality'), '[(]')[1].substr(1, 2))

# Mostrar as primeiras linhas para garantir que a coluna 'State' foi adicionada corretamente
df_agriculture_with_state.show(5)


+------+--------------------+----------+-----+
|  Year|        Municipality|Production|State|
+------+--------------------+----------+-----+
|1974.0|Alta Floresta D'O...|       0.0|   RO|
|1974.0|      Ariquemes (RO)|       0.0|   RO|
|1974.0|         Cabixi (RO)|       0.0|   RO|
|1974.0|         Cacoal (RO)|       0.0|   RO|
|1974.0|     Cerejeiras (RO)|       0.0|   RO|
+------+--------------------+----------+-----+
only showing top 5 rows



In [ ]:
# Filtrar o DataFrame para o período de 2004 a 2019
df_agriculture_filtered_period = df_agriculture_with_state.filter(
    (col('Year') >= 2004) & (col('Year') <= 2019)
)

# Exibir as primeiras linhas para garantir que o filtro foi aplicado corretamente
df_agriculture_filtered_period.show(5)


+------+--------------------+----------+-----+
|  Year|        Municipality|Production|State|
+------+--------------------+----------+-----+
|2004.0|Alta Floresta D'O...|       0.0|   RO|
|2004.0|      Ariquemes (RO)|       0.0|   RO|
|2004.0|         Cabixi (RO)|       0.0|   RO|
|2004.0|         Cacoal (RO)|       0.0|   RO|
|2004.0|     Cerejeiras (RO)|       0.0|   RO|
+------+--------------------+----------+-----+
only showing top 5 rows



**Aqui temos um gráfico apresentando quais estados mais produziram no período de 2004 - 2019**

In [ ]:
# Agrupar os dados por Estado e somar a produção no período 2004-2019
df_agriculture_production_by_state = df_agriculture_filtered_period.groupBy('State').sum('Production')

# Renomear a coluna de produção somada para 'Total_Production'
df_agriculture_production_by_state = df_agriculture_production_by_state.withColumnRenamed('sum(Production)', 'Total_Production')

# Coletar os dados do DataFrame Spark para um DataFrame Pandas
df_agriculture_production_by_state_pandas = df_agriculture_production_by_state.toPandas()

# Gráfico de barras com Plotly para exibir os estados que mais produziram no período 2004-2019
import plotly.express as px

fig = px.bar(
    df_agriculture_production_by_state_pandas,
    x='State',
    y='Total_Production',
    title='Produção Agrícola Total por Estado (2004-2019)',
    labels={'State': 'Estado', 'Total_Production': 'Produção Agrícola Total'},
    template='plotly_dark'
)

fig.update_layout(
    xaxis_title='Estado',
    yaxis_title='Produção Agrícola Total',
    xaxis=dict(tickangle=45)
)

fig.show()

A partir deste momento, após tratar os dados do nosso segundo dataset, começaremos de fato a trabalhar na correlação dos dois datasets.

In [ ]:
# Agrupar por Estado e Ano, somando a produção agrícola
df_agriculture_grouped = df_agriculture_filtered_period.groupBy('State', 'Year').sum('Production')

# Renomear a coluna de produção somada para 'Total_Production'
df_agriculture_grouped = df_agriculture_grouped.withColumnRenamed('sum(Production)', 'Total_Production')

# Exibir as primeiras linhas para verificar o agrupamento
df_agriculture_grouped.show(5)


+-----+------+----------------+
|State|  Year|Total_Production|
+-----+------+----------------+
|   SC|2005.0|       1272461.0|
|   RO|2015.0|        431491.0|
|   MA|2010.0|        958060.0|
|   AM|2006.0|         29381.0|
|   RJ|2014.0|          7139.0|
+-----+------+----------------+
only showing top 5 rows



In [ ]:
# Renomear a coluna 'Ano/Estados' para 'Year' no dataset de desmatamento e ajustar a coluna 'AMZ LEGAL'
df_deforestation_spark = df_deforestation_spark.withColumnRenamed('Ano/Estados', 'Year') \
                                               .withColumnRenamed('AMZ LEGAL', 'AMZ_LEGAL')

# Fazer o melt no dataset de desmatamento para que cada estado seja uma linha
df_deforestation_melted_spark = df_deforestation_spark.selectExpr(
    "Year",
    "stack(9, 'AC', AC, 'AM', AM, 'AP', AP, 'MA', MA, 'MT', MT, 'PA', PA, 'RO', RO, 'RR', RR, 'TO', TO) as (State, Deforestation)"
)

# Fazer o merge dos dois datasets com base em 'Year' e 'State'
df_merged = df_agriculture_grouped.join(df_deforestation_melted_spark, ['Year', 'State'])

# Exibir as primeiras linhas do DataFrame unido
df_merged.show(5)

# Calcular a correlação entre 'Total_Production' e 'Deforestation'
correlation = df_merged.stat.corr('Total_Production', 'Deforestation')

# Exibir a correlação
print(f"Correlação entre Produção Agrícola e Desmatamento: {correlation}")

+------+-----+----------------+-------------+
|  Year|State|Total_Production|Deforestation|
+------+-----+----------------+-------------+
|2015.0|   RO|        431491.0|         1030|
|2010.0|   MA|        958060.0|          712|
|2006.0|   AM|         29381.0|          788|
|2013.0|   AP|          8028.0|           23|
|2009.0|   RO|        331798.0|          482|
+------+-----+----------------+-------------+
only showing top 5 rows

Correlação entre Produção Agrícola e Desmatamento: 0.22652050764922943


O resultado da correlação entre a produção agrícola e o desmatamento foi de aproximadamente 0.23. Isso indica uma correlação positiva fraca entre os dois fatores, sugerindo que, à medida que a produção agrícola aumenta, o desmatamento também tende a aumentar, mas essa relação não é muito forte.

Vamos agora criar um gráfico de disperção para visualizarmos e interpretarmos melhor estes dados:

In [ ]:
# Coletar os dados do DataFrame Spark para um DataFrame Pandas
df_merged_pandas = df_merged.toPandas()

# Gráfico de dispersão com Plotly para visualizar a relação entre produção agrícola e desmatamento
import plotly.express as px

fig = px.scatter(
    df_merged_pandas,
    x='Total_Production',
    y='Deforestation',
    title='Relação entre Produção Agrícola e Desmatamento',
    labels={'Total_Production': 'Produção Agrícola (Total)', 'Deforestation': 'Área Desmatada (km²)'},
    template='plotly_dark'
)

fig.update_layout(
    xaxis_title='Produção Agrícola (Total)',
    yaxis_title='Área Desmatada (km²)',
)

fig.show()

In [ ]:
from pyspark.sql.functions import corr

# Calcular a correlação entre produção agrícola e desmatamento por estado
correlation_by_state = df_merged.groupBy('State').agg(corr('Total_Production', 'Deforestation').alias('Correlation'))

# Exibir as correlações por estado
correlation_by_state.show()

+-----+--------------------+
|State|         Correlation|
+-----+--------------------+
|   RO|-0.42220529018072395|
|   AM|  -0.393221441562285|
|   RR|  0.1828822928913394|
|   TO| -0.5421412327257331|
|   MT| -0.5225631950354467|
|   PA|-0.35580625301078933|
|   MA| -0.8093752179937127|
|   AC| 0.07166009730775776|
|   AP| -0.5528684090867174|
+-----+--------------------+



Os resultados da correlação por estado mostram variações interessantes na relação entre produção agrícola e desmatamento:

	•	Correlação positiva (fraca):
	•	AC (0.07) e RR (0.18): Esses estados mostram uma correlação positiva fraca, indicando que, à medida que a produção aumenta, o desmatamento também aumenta levemente, mas essa relação é bem fraca.
	•	Correlação negativa:
	•	AM (-0.39), AP (-0.55), MA (-0.81), MT (-0.52), PA (-0.35), RO (-0.42), TO (-0.54): Nesses estados, a correlação é negativa, o que sugere que, em muitos casos, o aumento da produção agrícola está associado a uma redução no desmatamento.
	•	Destaque para o Maranhão (MA): O estado de Maranhão apresenta uma correlação negativa muito forte (-0.81), o que indica que, nesse estado, o aumento da produção agrícola tem uma relação inversa muito clara com o desmatamento.

  Interpretação:

	•	Correlação negativa pode indicar que políticas ou práticas agrícolas mais eficientes estão em vigor em alguns estados, onde a produção agrícola aumenta sem causar um aumento proporcional no desmatamento.
	•	Correlação positiva em estados como Acre e Roraima pode refletir práticas agrícolas que ainda contribuem para o desmatamento, mesmo que de forma fraca.

 Vamos ver isso em um gráfico de disperção para termos melhor visualização desses insights:


In [ ]:
# Coletar os dados do DataFrame Spark para um DataFrame Pandas
df_correlation_by_state = correlation_by_state.toPandas()

# Gráfico de barras com Plotly para exibir a correlação por estado
import plotly.express as px

fig = px.bar(
    df_correlation_by_state,
    x='State',
    y='Correlation',
    title='Correlação entre Produção Agrícola e Desmatamento por Estado',
    labels={'State': 'Estado', 'Correlation': 'Correlação'},
    template='plotly_dark'
)

fig.update_layout(
    xaxis_title='Estado',
    yaxis_title='Correlação',
    xaxis=dict(tickangle=45)
)

fig.show()

**Análise da Relação entre Área Plantada e Desmatamento na Amazônia Legal (2004-2019)**

Nesta seção, expandimos nosso estudo para incluir dados de área plantada em diferentes estados da Amazônia Legal. O objetivo é investigar se há uma correlação entre o aumento da área plantada e o desmatamento. Abaixo estão os principais passos que realizaremos:

1. **Transformação da Estrutura de Dados**: O dataset de área plantada (`planted_area.csv`) está organizado com as colunas representando municípios. Para facilitar a análise, aplicamos uma técnica de transformação de dados (usando `stack`) que converte as colunas de municípios em linhas. Dessa forma, obtemos um formato de dados mais adequado para análise temporal e geográfica.

2. **Adição de Informações de Estado**: A partir do nome dos municípios (que contém a sigla do estado entre parênteses), extraímos a sigla para criar uma nova coluna chamada `State`. Isso nos permitirá agrupar a área plantada por estado e relacioná-la com os dados de desmatamento.

3. **Filtragem por Período de 2004 a 2019**: Para manter a consistência com o restante do estudo, filtramos o dataset para incluir apenas o período de 2004 a 2019.

4. **Agrupamento por Estado e Ano**: Somamos a área plantada total para cada estado em cada ano. Isso nos permitirá calcular a área total plantada por estado e ano, alinhando os dados para uma correlação precisa com o desmatamento.

5. **Integração com o Dataset de Desmatamento**: Realizamos o merge dos dados de área plantada com os dados de desmatamento, unindo-os pelas colunas `Year` e `State`. Isso nos dá um único dataset com informações de desmatamento e área plantada para cada estado e ano.

6. **Cálculo de Correlações e Visualização**: Calculamos a correlação entre a área plantada e o desmatamento para verificar a força e a direção dessa relação. Em seguida, visualizamos essa correlação em um gráfico de dispersão, o que nos permitirá avaliar melhor qualquer padrão ou tendência entre as variáveis.

Esses passos nos aproximam de uma análise mais aprofundada das relações entre atividades agrícolas e degradação ambiental, oferecendo insights sobre o impacto potencial do setor agrícola no desmatamento da região amazônica.

In [ ]:
#### 1. Empilhar as Colunas de Municípios
### - Semelhante ao dataset `harvested_area.csv`, vamos converter as colunas dos municípios em linhas para termos uma estrutura com `Municipality`, `Year`, e `Planted_Area`.

from pyspark.sql.functions import col

# Carregar o dataset 'planted_area.csv' no Spark
df_planted_area_spark = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/BigData/planted_area.csv', header=True, inferSchema=True)

# Coletar os nomes das colunas que representam os municípios (excluindo as colunas 'Grain' e 'Year')
municipality_columns = df_planted_area_spark.columns[2:]

# Converter todas as colunas de municípios para numérico (caso haja algum erro de tipo)
df_planted_area_spark = df_planted_area_spark.select(
    [col(c).cast("float").alias(c) if c in municipality_columns else col(c) for c in df_planted_area_spark.columns]
)

# Construir a expressão para a função stack, escapando os apóstrofos em nomes de municípios
def escape_apostrophe(municipality):
    return municipality.replace("'", "\\'")

stack_expr = "stack({}, {}) as (Municipality, Planted_Area)".format(
    len(municipality_columns),
    ', '.join([f"'{escape_apostrophe(municipality)}', `{municipality}`" for municipality in municipality_columns])
)

# Aplicar a função stack para transformar colunas em linhas
df_planted_area_melted_spark = df_planted_area_spark.selectExpr("Year", stack_expr)

# Filtrar as linhas onde o Município não é 'Year'
df_planted_area_melted_spark = df_planted_area_melted_spark.filter(df_planted_area_melted_spark.Municipality != "Year")

# Mostrar algumas linhas para verificar o resultado
df_planted_area_melted_spark.show(5)


#### 2. Adicionar a Coluna de `State`
### Usamos a mesma técnica para extrair a sigla do estado dos nomes dos municípios.

from pyspark.sql.functions import split

# Adicionar a coluna de Estado extraindo a sigla dos nomes dos municípios
df_planted_area_with_state = df_planted_area_melted_spark.withColumn('State', split(col('Municipality'), '[(]')[1].substr(1, 2))

# Verificar se a coluna 'State' foi adicionada corretamente
df_planted_area_with_state.show(5)


#### 3. Filtrar para o Período de 2004 a 2019
### Mantemos a consistência com os demais datasets.


# Filtrar para o período de 2004 a 2019
df_planted_area_filtered_period = df_planted_area_with_state.filter(
    (col('Year') >= 2004) & (col('Year') <= 2019)
)

# Exibir as primeiras linhas para confirmar o filtro
df_planted_area_filtered_period.show(5)


#### 4. Agrupar por Estado e Ano
### Agrupamos a área plantada total por estado e ano.


# Agrupar por Estado e Ano, somando a área plantada
df_planted_area_grouped = df_planted_area_filtered_period.groupBy('State', 'Year').sum('Planted_Area')

# Renomear a coluna para 'Total_Planted_Area'
df_planted_area_grouped = df_planted_area_grouped.withColumnRenamed('sum(Planted_Area)', 'Total_Planted_Area')

# Exibir as primeiras linhas para verificar o agrupamento
df_planted_area_grouped.show(5)


#### 5. Realizar o Merge com os Dados de Desmatamento
### Usamos o dataset `df_deforestation_melted_spark` para unir as tabelas com base em `State` e `Year`, adicionando a área plantada ao conjunto de dados para correlações.


# Fazer o merge dos dados de área plantada com o dataset de desmatamento
df_planted_deforestation_merged = df_planted_area_grouped.join(df_deforestation_melted_spark, ['Year', 'State'])

# Exibir algumas linhas do DataFrame unido para confirmação
df_planted_deforestation_merged.show(5)


#### 6. Calcular Correlações e Visualizar
### Calculamos a correlação entre a área plantada e o desmatamento e geramos gráficos com Plotly para visualização.


# Calcular a correlação entre 'Total_Planted_Area' e 'Deforestation'
correlation_planted_deforestation = df_planted_deforestation_merged.stat.corr('Total_Planted_Area', 'Deforestation')
print(f"Correlação entre Área Plantada e Desmatamento: {correlation_planted_deforestation}")

# Converter para Pandas e gerar gráfico com Plotly
df_planted_deforestation_merged_pandas = df_planted_deforestation_merged.toPandas()

import plotly.express as px

fig = px.scatter(
    df_planted_deforestation_merged_pandas,
    x='Total_Planted_Area',
    y='Deforestation',
    title='Relação entre Área Plantada e Desmatamento',
    labels={'Total_Planted_Area': 'Área Plantada Total', 'Deforestation': 'Área Desmatada (km²)'},
    template='plotly_dark'
)
fig.show()

+------+--------------------+------------+
|  Year|        Municipality|Planted_Area|
+------+--------------------+------------+
|1974.0|Alta Floresta D'O...|         0.0|
|1974.0|      Ariquemes (RO)|         0.0|
|1974.0|         Cabixi (RO)|         0.0|
|1974.0|         Cacoal (RO)|         0.0|
|1974.0|     Cerejeiras (RO)|         0.0|
+------+--------------------+------------+
only showing top 5 rows

+------+--------------------+------------+-----+
|  Year|        Municipality|Planted_Area|State|
+------+--------------------+------------+-----+
|1974.0|Alta Floresta D'O...|         0.0|   RO|
|1974.0|      Ariquemes (RO)|         0.0|   RO|
|1974.0|         Cabixi (RO)|         0.0|   RO|
|1974.0|         Cacoal (RO)|         0.0|   RO|
|1974.0|     Cerejeiras (RO)|         0.0|   RO|
+------+--------------------+------------+-----+
only showing top 5 rows

+------+--------------------+------------+-----+
|  Year|        Municipality|Planted_Area|State|
+------+----------------

### Análise da Relação entre Focos de Incêndio, Área Plantada e Desmatamento na Amazônia Legal

Para enriquecer nossa análise sobre os fatores que contribuem para a degradação da Amazônia Legal, integramos dados sobre **focos de incêndio** registrados em cada estado. O objetivo é explorar como os incêndios se relacionam com o desmatamento e com as atividades agrícolas. Abaixo estão os passos que vamos seguir:

1. **Agrupamento dos Focos de Incêndio por Estado e Ano**:
   - O dataset de incêndios (`inpe_brazilian_amazon_fires_1999_2019.csv`) fornece informações sobre o número de focos de incêndio por mês, estado e localização. Para simplificar a análise, agrupamos esses dados por **estado** e **ano**, calculando o total de focos de incêndio para cada combinação de ano e estado.

2. **Padronização dos Nomes das Colunas**:
   - Renomeamos as colunas `year` e `state` para `Year` e `State`, mantendo a consistência com os outros datasets e facilitando a integração entre eles.

3. **Integração dos Dados de Incêndio com o DataFrame Principal**:
   - Realizamos um merge do dataset de incêndio com o conjunto de dados já unido (que contém informações sobre **área plantada** e **desmatamento**), usando as colunas `Year` e `State` como chave. Isso nos permite analisar todos os fatores em um único DataFrame.

4. **Cálculo das Correlações e Visualização**:
   - Calculamos a correlação entre o total de focos de incêndio e as variáveis `Deforestation` e `Total_Planted_Area`. Isso nos dará uma medida quantitativa da relação entre incêndios, desmatamento e atividades agrícolas.
   - Em seguida, geramos gráficos de dispersão que mostram visualmente a relação entre focos de incêndio e desmatamento, ajudando a identificar padrões e tendências.

Essa integração dos dados de incêndio com as demais variáveis oferece uma visão mais abrangente sobre os fatores ambientais e humanos que afetam a Amazônia Legal, possibilitando uma análise mais completa e informada das relações entre atividades agrícolas, incêndios e desmatamento.

In [ ]:
### Estrutura do Dataset
# **Entradas**: 2104
# **Colunas**: 6, que são:
  # **year** (int64): Ano do registro dos focos de incêndio.
  # **month** (int64): Mês do registro.
  # **state** (object): Nome do estado onde o foco de incêndio foi registrado.
  # **latitude** (float64): Latitude do ponto de incêndio.
  # **longitude** (float64): Longitude do ponto de incêndio.
  # **firespots** (int64): Número de focos de incêndio registrados no local.

### Observações
# Esse dataset oferece uma visão geoespacial e temporal dos focos de incêndio em cada estado brasileiro.
# A coluna `firespots` pode ser agregada por **estado e ano** para relacionar o número de focos de incêndio com o desmatamento e a área plantada, como fizemos nos datasets anteriores.

### Próximos Passos
# Para integrar esse dataset ao notebook, podemos realizar as seguintes etapas:

    #1. **Agrupar Focos de Incêndio por Estado e Ano**:
    #   - Calcularemos o total de focos de incêndio (`firespots`) para cada estado em cada ano.

    #2. **Realizar o Merge com os Dados de Desmatamento e Área Plantada**:
    #   - Usaremos `state` e `year` como chaves para combinar com o dataset de desmatamento e área plantada, a fim de explorar possíveis correlações.

    #3. **Calcular e Visualizar as Correlações**:
    #   - Verificaremos se há correlação entre os focos de incêndio e os dados de desmatamento e área plantada.

#### 1. Agrupar os Dados de Incêndio

# Carregar o dataset de incêndios no Spark
df_fires_spark = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/BigData/inpe_brazilian_amazon_fires_1999_2019.csv', header=True, inferSchema=True)

# Agrupar por estado e ano, somando os focos de incêndio
from pyspark.sql.functions import col, upper

df_fires_grouped = (
    df_fires_spark.groupBy('year', 'state')
    .sum('firespots')
    .withColumnRenamed('sum(firespots)', 'Total_Firespots')
)

# Ajustar o Nome das Colunas para Consistência
df_fires_grouped = (
    df_fires_grouped.withColumnRenamed('year', 'Year')
    .withColumnRenamed('state', 'State')
)

# Dicionário para converter nomes completos dos estados para abreviações
state_abbr = {
    "RONDONIA": "RO", "AMAZONAS": "AM", "RORAIMA": "RR", "TOCANTINS": "TO",
    "MATO GROSSO": "MT", "PARA": "PA", "MARANHAO": "MA", "ACRE": "AC", "AMAPA": "AP"
}

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Função UDF para converter o nome completo do estado em abreviação
def convert_state_name(state):
    return state_abbr.get(state, state)  # Retorna o nome original se não estiver no dicionário

convert_state_udf = udf(convert_state_name, StringType())
df_fires_grouped = df_fires_grouped.withColumn("State", convert_state_udf(col("State")))

# Converter o ano em df_planted_deforestation_merged para inteiro
df_planted_deforestation_merged = df_planted_deforestation_merged.withColumn("Year", col("Year").cast("int"))

# Fazer o Merge dos Dados de Incêndio com os Dados Existentes
df_final_merged = df_planted_deforestation_merged.join(df_fires_grouped, ['Year', 'State'], how='left')

# Exibir algumas linhas para confirmar o merge
df_final_merged.show(5)

# Calcular as Correlações e Visualizar
correlation_fires_desa = df_final_merged.stat.corr('Total_Firespots', 'Deforestation')
correlation_fires_plant = df_final_merged.stat.corr('Total_Firespots', 'Total_Planted_Area')

print(f"Correlação entre Focos de Incêndio e Desmatamento: {correlation_fires_desa}")
print(f"Correlação entre Focos de Incêndio e Área Plantada: {correlation_fires_plant}")

# Visualizar a relação entre focos de incêndio e desmatamento
import plotly.express as px
df_final_merged_pandas = df_final_merged.toPandas()

fig = px.scatter(
    df_final_merged_pandas,
    x='Total_Firespots',
    y='Deforestation',
    title='Relação entre Focos de Incêndio e Desmatamento',
    labels={'Total_Firespots': 'Focos de Incêndio', 'Deforestation': 'Área Desmatada (km²)'},
    template='plotly_dark'
)
fig.show()

# Verificar os valores únicos de 'Year' e 'State' em cada DataFrame para garantir que coincidem

# Valores únicos de 'State' no dataset de incêndios
print("Valores únicos de State em df_fires_grouped:")
df_fires_grouped.select("State").distinct().show()

# Valores únicos de 'State' no dataset de área plantada e desmatamento
print("Valores únicos de State em df_planted_deforestation_merged:")
df_planted_deforestation_merged.select("State").distinct().show()

# Valores únicos de 'Year' em ambos os datasets
print("Valores únicos de Year em df_fires_grouped:")
df_fires_grouped.select("Year").distinct().show()

print("Valores únicos de Year em df_planted_deforestation_merged:")
df_planted_deforestation_merged.select("Year").distinct().show()


+----+-----+------------------+-------------+---------------+
|Year|State|Total_Planted_Area|Deforestation|Total_Firespots|
+----+-----+------------------+-------------+---------------+
|2015|   RO|          431842.0|         1030|          13105|
|2010|   MA|          968310.0|          712|           7298|
|2006|   AM|           29629.0|          788|          11697|
|2013|   AP|            8096.0|           23|           1529|
|2009|   RO|          331798.0|          482|           5396|
+----+-----+------------------+-------------+---------------+
only showing top 5 rows

Correlação entre Focos de Incêndio e Desmatamento: 0.9060224623062638
Correlação entre Focos de Incêndio e Área Plantada: 0.19512106510797922


Valores únicos de State em df_fires_grouped:
+-----+
|State|
+-----+
|   RO|
|   AM|
|   RR|
|   TO|
|   MT|
|   PA|
|   MA|
|   AC|
|   AP|
+-----+

Valores únicos de State em df_planted_deforestation_merged:
+-----+
|State|
+-----+
|   RO|
|   AM|
|   RR|
|   TO|
|   MT|
|   PA|
|   MA|
|   AC|
|   AP|
+-----+

Valores únicos de Year em df_fires_grouped:
+----+
|Year|
+----+
|2003|
|2007|
|2018|
|2015|
|2006|
|2013|
|2014|
|2019|
|2004|
|2012|
|2009|
|2016|
|2001|
|2005|
|2000|
|2010|
|2011|
|2008|
|2017|
|1999|
+----+
only showing top 20 rows

Valores únicos de Year em df_planted_deforestation_merged:
+----+
|Year|
+----+
|2007|
|2018|
|2015|
|2006|
|2013|
|2014|
|2019|
|2004|
|2012|
|2009|
|2016|
|2005|
|2010|
|2011|
|2008|
|2017|
+----+



### Insights Finais e Encerramento do Projeto

Ao longo deste projeto, exploramos dados de desmatamento, produção agrícola e focos de incêndio na região amazônica brasileira, buscando compreender as relações entre esses fatores e a degradação da floresta. Com a integração e análise desses dados, chegamos a importantes conclusões:

1. **Incêndios e Desmatamento**:
   - A análise revelou uma **forte correlação positiva** entre o número de focos de incêndio e a área desmatada, com um coeficiente de correlação de aproximadamente **0.91**. Esse resultado indica que, na Amazônia Legal, o aumento de focos de incêndio está diretamente associado ao aumento das áreas desmatadas, reforçando a hipótese de que incêndios são um dos principais fatores impulsionadores do desmatamento.

2. **Incêndios e Área Plantada**:
   - Observamos uma **correlação fraca e positiva** (cerca de **0.20**) entre os focos de incêndio e a área plantada. Embora exista uma leve tendência de incêndios em regiões com maior extensão de áreas plantadas, essa relação é consideravelmente menos intensa quando comparada com a relação entre incêndios e desmatamento. Isso sugere que, embora a agricultura possa contribuir indiretamente para a ocorrência de incêndios, outros fatores parecem estar mais fortemente relacionados ao aumento dos focos de incêndio e à degradação da floresta.

3. **Impacto Geral**:
   - Esse estudo reforça a importância de políticas públicas focadas na prevenção de incêndios como uma medida essencial para a preservação da floresta amazônica. As áreas desmatadas não só contribuem para a perda de biodiversidade como também intensificam as emissões de gases de efeito estufa, afetando diretamente o equilíbrio ambiental do planeta.

Gostaríamos de expressar nossa profunda gratidão ao professor **Dr. Sebastião Rogério**, cuja orientação e conhecimentos foram fundamentais para o desenvolvimento deste projeto. A todos que acompanharam nossa jornada até aqui, nossos sinceros agradecimentos. Esperamos que as descobertas apresentadas inspirem a busca por soluções eficazes para a proteção da nossa Amazônia e o desenvolvimento sustentável da região.

**Obrigado a todos!**